# TPS example

### Estimate scaling Factor 
Call gammasim with no noise data, only 1 peak

In [1]:
from configuration.model.config_model import load_config
import sys
print(sys.path)
sys.path.append("mnt/Windows/SHARED/ws_vscode/gammalib")
sys.path.append("/mnt/Windows/SHARED/ws_vscode/gammalib/area_eval/")
from gammasim import GammaSim
from implementation.algorithm import TrapezoidalShaperAlg
from implementation.filt_param_handlers import read_trap_params

 # Carica la configurazione
config_file_name = "config_tps_config_method2-no-noise.json"
cfg = load_config(config_file_name)
print(f"Configuration imported! \n {cfg} \n\n")

# Istanzio il simulatore GammaSim
config_file = cfg.gammasim_cfg
saturation = False
gammasim = GammaSim(config_file)
gammasim.generate_dataset(saturation)

# Crea l'istanza dell'algoritmo e calcola i risultati
ds = gammasim.get_dataset()
M = gammasim.get_params()[0][0]['tau2']
sampling_time=gammasim.get_sampling_time()
alg = TrapezoidalShaperAlg(dataset=ds, config=cfg, sampling_time=sampling_time, M=M)

csv_file="notebook_values.cfg"
trap_heights = alg.find_area_gain(gammasim.get_areas(), out=csv_file)
print(read_trap_params(csv_file))

['/home/luca', '/mnt/Windows/SHARED/ws_vscode/gammalib/gammasim', '/mnt/Windows/SHARED/ws_vscode/gammalib/area_eval', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luca/venvs/gammasim/lib/python3.10/site-packages']
Configuration imported! 
 gammasim_cfg='config_method2-no-noise.json' time_filter=TimeFilterParams(alpha_l=0.002, alpha_h=0.9, gain_k=0.0001, in_cond=[100.0], th_dy=15432.336840189751, th_d2y=48515526534.46768, dev_ord_det=1, zero_cr_window=10, window_sz=15) trap_filter=TrapParams(m=110, l=30, ftd_s=20, ftd_e=0, int_w=10) 


STEP 1: Generating the number of peaks for each sample
start_time: 1731606885.254243
stop_time:  1731606885.254415, total time for this step = 0.00017166

STEP 2: Generating parameters for each peak
start_time: 1731606885.254426
stop_time:  1731606885.462118, total time for this step = 0.20769215

STEP 3: Generating signals for each peak
start_time: 1731606885.462168


  0%|          | 0/10000 [00:00<?, ?it/s]

stop_time:  1731606886.261191, total time for this step = 0.79902291

STEP 4: Generating labels
start_time: 1731606886.261263
stop_time:  1731606886.405518, total time for this step = 0.14425516

STEP 5: Applying noise to dataset
start_time: 1731606886.405574
stop_time:  1731606887.141679, total time for this step = 0.73610473

TOTAL TIME FOR GENERATE DATASET = 1.88753247

(10000, 2000)
(2000,)
2000
ord detection:  1
candidate_peaks:  (array([561]), {'peak_heights': array([27833.76983922]), 'prominences': array([36956.09656633]), 'left_bases': array([0]), 'right_bases': array([613])})
t0 detected: :  [561]
window in :  [611, 701]
height: 5899.615683337442, sim area:179985.46999839356
(2000,)
2000
ord detection:  1
candidate_peaks:  (array([862]), {'peak_heights': array([27833.76983922]), 'prominences': array([36956.09656633]), 'left_bases': array([0]), 'right_bases': array([914])})
t0 detected: :  [862]
window in :  [912, 1002]
height: 5899.593318144254, sim area:179985.46999839353
(20

visualize saved data

In [ ]:
# Visualizza i risultati
print(trap_heights)
alg.plot_results()


### Estimate trigger threshold on noisy data